## Set Up Dependencies

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp


## Fetch and Preprocess Data

In [ ]:
df = pd.read_parquet("https://osf.io/trz42/download")


In [ ]:
df = df.rename(
    columns={
        "annotation_size_bits": "Annotation Size (bits)",
        "algorithm": "Algorithm",
        "population_size": "Population Size",
        "num_generations": "Num Generations",
        "num_islands": "Num Islands",
        "num_niches": "Num Niches",
        "tournament_size": "Tournament Size",
        "replicate": "Replicate"
    },
)


In [ ]:
df["Dynamics"] = (
    "islands"
    + df["Num Islands"].astype(str)
    + "-niches"
    + df["Num Niches"].astype(str)
    + "-tsize"
    + df["Tournament Size"].astype(str)
)


In [ ]:
df["Scenario"] = df["Dynamics"].map(
    {
        "islands1-niches1-tsize2": "plain",
        "islands1-niches1-tsize1": "drift",
        "islands4-niches2-tsize2": "spatial/ecological mild",
        "islands64-niches8-tsize2": "spatial/ecological strong",
    }
)

In [ ]:
df["algo"] = df["Algorithm"].map(
    {
        "new-steady": "new\nsteady",
        "old-steady": "old\nsteady",
        "new-hybrid": "new\nhybrid",
        "new-tilted": "new\ntilted",
        "old-tilted": "old\ntilted",
    },
)

In [ ]:
df["Scale"] = (
    "npop"
    + df["Population Size"].astype(str)
    + "-ngen"
    + df["Num Generations"].astype(str)
)

In [ ]:
row_order=[
    # "new-steady",
    # "old-steady",
    # "new-hybrid",
    # "new-tilted",
    # "old-tilted",
    "new\nsteady",
    "old\nsteady",
    "new\nhybrid",
    "new\ntilted",
    "old\ntilted",
]

def joyhist(*args, **kwargs):
    sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

    g = sns.displot(
        *args,
        **kwargs,
        kind="hist",
        row_order=row_order,
        hue_order=["true", "reconstructed"],
        multiple="fill",
        common_norm=False,
        aspect=3,
        height=1,
        log_scale=(True, False),
        bins=20,
        element="step",
        palette=["none", "#1f77b4"],
        facet_kws=dict(margin_titles=True),
        legend=False,
    )
    g.refline(y=0.5, linestyle="--", linewidth=2, color="black")
    g.refline(y=0.0, linestyle="-", linewidth=3, color="black")

    g.figure.subplots_adjust(hspace=-0.4)

    g.set_titles("")
    g.set(yticks=[], ylabel="")
    g.despine(bottom=True, left=True)
    g.set_titles(col_template="{col_name}", row_template="{row_name}")

    for r, arr in enumerate(g.axes):
        for c, ax in enumerate(arr):
            color = plt.get_cmap('tab10')(r)
            for patch in ax.get_children():
                try:
                    if isinstance(patch, mpl.collections.PolyCollection):
                        patch.set_facecolor(color)
                        patch.set_alpha(0.5)
                        break
                except AttributeError:
                    pass
    return g

In [ ]:
for __, group in df.groupby("Scale"):
    group["Time Ago"] = group["origin_time"].max() + 1 - group["origin_time"]
    tp.tee(
        joyhist,
        data=group,
        x="Time Ago",
        row="algo",
        col="Scenario",
        hue="kind",
    )

## Reproducibility

In [ ]:
import datetime
datetime.datetime.now().isoformat()

In [ ]:
%load_ext watermark
%watermark

In [ ]:
!pip freeze